<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-exploration-of-download-matches" data-toc-modified-id="Data-exploration-of-download-matches-1">Data exploration of download matches</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1">Imports</a></span></li><li><span><a href="#Exploration" data-toc-modified-id="Exploration-1.2">Exploration</a></span></li><li><span><a href="#Csv-file-creation" data-toc-modified-id="Csv-file-creation-1.3">Csv file creation</a></span><ul class="toc-item"><li><span><a href="#Single-agent" data-toc-modified-id="Single-agent-1.3.1">Single agent</a></span></li><li><span><a href="#Best-n-agents" data-toc-modified-id="Best-n-agents-1.3.2">Best n agents</a></span></li></ul></li></ul></li></ul></div>

# Data exploration of download matches

## Imports

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Exploration

In [3]:
df = pd.read_csv('/home/gbarbadillo/luxai_ssd/agent_selection.csv')
df.sort_values('FinalScore', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore,FinalScore
0,69945543,27424471,1,90009.0,2,23032370,36.889864,1560.389280,36.522500,1566.517103,1818.288755
1,69923394,27413397,0,650053.0,2,23032370,38.435234,1536.093066,38.058785,1541.342982,1818.288755
2,69849883,27376641,1,410038.0,2,23032370,87.761971,1111.109255,83.368953,1142.533822,1818.288755
3,69847811,27375605,1,150010.0,2,23032370,144.366002,871.859256,135.552630,896.074161,1818.288755
4,69847037,27375218,1,130011.0,2,23032370,185.000000,702.140727,170.000000,788.476153,1818.288755


In [4]:
len(df.SubmissionId.unique())

56

In [5]:
summary = pd.DataFrame([
    df.groupby('SubmissionId').count().max(axis=1),
    df.groupby('SubmissionId').max()['FinalScore']]).transpose()
summary.sort_values('FinalScore', ascending=False).head(10)

,Unnamed 0,FinalScore
SubmissionId,,
23032370,176.0,1818.288755
22777661,394.0,1711.493722
23034455,244.0,1710.606181
22934122,241.0,1651.606058
22902113,345.0,1650.975344
23159502,65.0,1650.841907
23038485,137.0,1643.818123
23010508,195.0,1643.516025
23008895,191.0,1636.605564


This can be used to choose how many matches I want to use for training.

## Csv file creation

### Single agent

In [10]:
for idx, submission_id in enumerate(df.SubmissionId.unique()[:5]):
    sub_df = df[df.SubmissionId == submission_id]
    filepath = '/mnt/hdd0/Kaggle/luxai/models/16_train_on_single_agent/'
    filepath += 'rank%i_id%i_score%i.csv' % (idx, submission_id, sub_df['FinalScore'].max())
    sub_df.to_csv(filepath, index=False)
    print(idx, len(sub_df))

0 176
1 394
2 244
3 241
4 345


### Best n agents

In [13]:
for n_agents in range(2, 6):
    submission_ids = set(df.SubmissionId.unique()[:n_agents])
    sub_df = df[df.SubmissionId.apply(lambda x: x in submission_ids)]
    filepath = '/mnt/hdd0/Kaggle/luxai/models/17_train_on_best_n_agents/'
    filepath += 'best_%i.csv' % (n_agents)
    sub_df.to_csv(filepath, index=False)
    print(n_agents, len(sub_df))

2 570
3 814
4 1055
5 1400
